In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<style>.container { width:85% !important; }</style>
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
'''))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import rcParams as rc
from pprint import pprint
from gdax_helpers import *
from api_key_sb import *

In [ ]:
client = gdax.AuthenticatedClient(KEY,
                                       B64SECRET,
                                       PASSPHRASE)
dfhist = get_history_df(client, get_account_df(client))
res = get_portfolio_history(client, dfhist)

In [ ]:
rc.update({'font.size': 18})
size=(23,13)

# Principal Graph

In [ ]:
y1 = res['principal']['principal']
y2 = res['total']['close']
x = y1.index.to_pydatetime()

fig, ax = plt.subplots(figsize=size)
ax.grid(True)
ax.plot(x, y1)
ax.plot(x, y2, 'k')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax.legend(['Principal', 'Portfolio'], loc='lower right')
ax.set_title('Portfolio Gain')
ax.set_ylim(0, ax.get_ylim()[1] + 100)
ax.set_ylabel('Total Value [$]')

ax.fill_between(
    x=x,
    y1=y1.values,
    y2=y2.values,
    where=y2.values <= y1.values,
    facecolor='red'
)

ax.fill_between(
    x=x,
    y1=y1.values,
    y2=y2.values,
    where=y2.values >= y1.values,
    facecolor='green'
)

val = y2.subtract(y1, axis=0).max()
i = y2[y2.subtract(y1, axis=0) == val].index[0]
ax.annotate('${:.2f}\n{}'.format(y2.loc[i],
                                 i.strftime('%m/%d %H:%M')),
            xy=(i, y2.loc[i]),
            xycoords='data',
            xytext=(-150, 35),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))

val = y2.subtract(y1, axis=0).min()
i = y2[y2.subtract(y1, axis=0) == val].index[0]
ax.annotate('${:.2f}\n{}'.format(y2.loc[i],
                                 i.strftime('%m/%d %H:%M')),
            xy=(i, y2.loc[i]),
            xycoords='data',
            xytext=(50, -30),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))

for i, row in dfhist.drop('USD', level=0).iterrows():
    x = i[1]
    y = y1[y1.index > x]
    y = y[0] / ax.get_ylim()[1]
    ax.axvline(x, ymax=y)

In [ ]:
y1 = res['principal']['principal']
y2 = res['total']['close']
y3 = res['ETH-USD']['close']
y4 = res['BTC-USD']['close']
y5 = res['LTC-USD']['close']
x = y1.index.to_pydatetime()

fig, ax = plt.subplots(figsize=size)
ax.grid(True)
ax.plot(x, y1)
ax.plot(x, y2)
ax.plot(x, y3)
ax.plot(x, y4)
ax.plot(x, y5)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax.legend(['Principal', 'Total', 'ETH', 'BTC', 'LTC'], loc='upper left')
ax.set_title('Portfolio Gain')
ax.set_ylim(0, ax.get_ylim()[1] + 100)
ax.set_ylabel('Total Value [$]')

# Portfolio Rate

In [ ]:
data = res['total']['close'].divide(res['principal']['principal'], axis=0) - 1
data *= 100
fig, ax = plt.subplots(figsize=size)
ax.grid(True)
ax.plot(data.index.to_pydatetime(), data.values)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax.set_title('Portfolio Gain')
ax.set_ylabel('Return [%]')

y1 = data.values
y2 = np.full((len(data.values)), 0)
ax.fill_between(
    x=data.index,
    y1=y1,
    y2=y2,
    where=y2 <= y1,
    facecolor='green'
)

ax.fill_between(
    x=data.index,
    y1=y1,
    y2=y2,
    where=y2 >= y1,
    facecolor='red'
)

ax.annotate('{:.2f}%\n{}'.format(data.max(),
                                 data[data == data.max()].index[0].strftime('%m/%d %H:%M')),
            xy=(data[data == data.max()].index[0], data.max()),
            xycoords='data',
            xytext=(-150, -20),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))

ax.annotate('{:.2f}%\n{}'.format(data.min(),
                                 data[data == data.min()].index[0].strftime('%m/%d %H:%M')),
            xy=(data[data == data.min()].index[0], data.min()),
            xycoords='data',
            xytext=(50, 10),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))

# Portfolio Gain

In [ ]:
data = res['total']['close'].subtract(res['principal']['principal'], axis=0)
fig, ax = plt.subplots(figsize=size)
ax.grid(True)
ax.plot(data.index.to_pydatetime(), data.values)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax.set_title('Portfolio Gain')
# ax.set_ylim(-100, 100)
ax.set_ylabel('Return [$]')
y1 = data.values
y2 = np.full((len(data.values)), 0)
ax.fill_between(
    x=data.index,
    y1=y1,
    y2=y2,
    where=y2 <= y1,
    facecolor='green'
)

ax.fill_between(
    x=data.index,
    y1=y1,
    y2=y2,
    where=y2 >= y1,
    facecolor='red'
)


ax.annotate('${:.2f}\n{}'.format(data.max(),
                                 data[data == data.max()].index[0].strftime('%m/%d %H:%M')),
            xy=(data[data == data.max()].index[0], data.max()),
            xycoords='data',
            xytext=(-150, -20),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))

ax.annotate('${:.2f}\n{}'.format(data.min(),
                             data[data == data.min()].index[0].strftime('%m/%d %H:%M')),
            xy=(data[data == data.min()].index[0], data.min()),
            xycoords='data',
            xytext=(50, -10),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->'))